In [1]:
import ar1_approx as ar1
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
import scipy.integrate as integrate
import scipy.optimize as opt
import time
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import numba

In [20]:
import ar1_approx as ar1

In [2]:
# Define a function to get aggregate value
@numba.jit
def Agg(matA, matB):
    aggregate = (np.multiply(matA, matB)).sum()
    return aggregate

In [3]:
@numba.jit
def SD_loop(PF, Gamma, sizez, sizek):
    HGamma = np.zeros((sizez, sizek))
    for i in range(sizez):  # z
        for j in range(sizek):  # k
            for m in range(sizez):  # z'
                HGamma[m, PF[i, j]] = \
                    HGamma[m, PF[i, j]] + pi[i, m] * Gamma[i, j]
    return HGamma

In [4]:
# Simulate the Markov process - will make this a function so can call later
@numba.jit
def sim_markov(z_grid, pi, num_draws):
    # draw some random numbers on [0, 1]
    u = np.random.uniform(size=num_draws)

    # Do simulations
    z_discrete = np.empty(num_draws)  # this will be a vector of values 
    # we land on in the discretized grid for z
    N = z_grid.shape[0]
    oldind = int(np.ceil((N - 1) / 2)) # set initial value to median of grid
    z_discrete[0] = z_grid[oldind]  
    for i in range(1, num_draws):
        sum_p = 0
        ind = 0
        while sum_p < u[i]:
            sum_p = sum_p + pi[ind, oldind]
#             print('inds =  ', ind, oldind)
            ind += 1
        if ind > 0:
            ind -= 1
        z_discrete[i] = z_grid[ind]
        oldind = ind
                            
    return z_discrete

In [5]:
# Use Rouwenhorst (1995) method
rho = 0.7605
mu = 0.0
N = 9
num_draws = 100000 # number of shocks to draw
sigma_eps = 0.2
sigma_z = sigma_eps / ((1 - rho ** 2) ** (1 / 2))
num_sigma = 4
alpha_k = 0.297
alpha_l = 0.650
delta = 0.154
psi = 1.08
r = 0.04
h = 6.616
betafirm = (1 / (1 + r))
step = (num_sigma * sigma_z) / (N / 2)
w = 0.7

In [6]:
# Call rouwen function to get z grids
pi, z_grid = ar1.rouwen(rho, mu, step, N)
z_grid = np.exp(z_grid)

In [7]:
# Define kgrid function

In [8]:
@numba.jit
def kgrid_func(w):
    """
    Operating profits function
    """
    z = 1
    ### Get grid points for k (capital stock)
    dens = 5
    # put in bounds here for the capital stock space
    kstar = ((((1 / betafirm - 1 + delta) * ((w / alpha_l) ** (alpha_l / (1 - alpha_l)))) /
             (alpha_k * (z ** (1 / (1 - alpha_l))))) **
             ((1 - alpha_l) / (alpha_k + alpha_l - 1)))
    kbar = 2*kstar
    lb_k = 0.001
    ub_k = kbar
    krat = np.log(lb_k / ub_k)
    numb = np.ceil(krat / np.log(1 - delta))
    K = np.zeros(int(numb * dens))
    # we'll create in a way where we pin down the upper bound - since
    # the distance will be small near the lower bound, we'll miss that by little

    for j in range(int(numb * dens)):
        K[j] = ub_k * (1 - delta) ** (j / dens)
    kgrid = K[::-1]
    return kgrid

In [9]:
# Define operating profits

In [10]:
@numba.jit
def op_profits(w):
    """
    Operating profits function
    """
    # operating profits, op
    kgrid = kgrid_func(w)
    sizek = kgrid.shape[0]
    sizez = z_grid.shape[0]
    op = np.zeros((sizez, sizek))
 
    for i in range(sizez):
            for j in range(sizek):
                op[i,j] = ((1 - alpha_l) * ((alpha_l / w) ** (alpha_l / (1 - alpha_l))) *
              ((kgrid[j] ** alpha_k) ** (1 / (1 - alpha_l))) * (z_grid[i] ** (1/(1 - alpha_l))))
    
    return op

In [11]:
# Define cost function

In [12]:
@numba.jit
def adj_cost(w):
    """
    Adjustment cost function
    """
    kgrid = kgrid_func(w)
    sizek = kgrid.shape[0]
    cost = np.zeros((sizek, sizek))

    for i in range(sizek):
        for j in range(sizek):
            cost[i, j] = (psi / 2) * ((kgrid[j] - ((1 - delta) * kgrid[i])) ** 2)/ kgrid[i]
    
    return cost

In [13]:
# Define cash flow

In [14]:
@numba.jit
def cash_flow(w):
    kgrid = kgrid_func(w)
    sizek = kgrid.shape[0]
    sizez = z_grid.shape[0]
    op = op_profits(w)
    cost = adj_cost(w)
    
    e = np.zeros((sizez, sizek, sizek))
    for i in range(sizez):
        for j in range(sizek):
            for k in range(sizek):
                e[i, j, k] = (op[i,j] - kgrid[k] + ((1 - delta) * kgrid[j]) - cost[k, j])
    return e

In [18]:
# Define a function of VF_loop
#@numba.jit
def VFI_loop(w):
    e = cash_flow(w)
    kgrid = kgrid_func(w)
    sizek = kgrid.shape[0]
    sizez = z_grid.shape[0] 
    
    VFtol = 1e-6
    VFdist = 7.0
    VFmaxiter = 3000
    V = np.zeros((sizez, sizek))  # initial guess at value function
    Vmat = np.zeros((sizez, sizek, sizek))  # initialize Vmat matrix
    Vstore = np.zeros((sizez, sizek, VFmaxiter))  # initialize Vstore array
    VFiter = 1
    
    while VFdist > VFtol and VFiter < VFmaxiter:
        TV = V        
        V_prime = np.dot(pi, V)
        for i in range(sizez):  # loop over z
            for j in range(sizek):  # loop over k
                for k in range(sizek): # loop over k'
                    Vmat[i, j, k] = e[i, j, k] + betafirm * V_prime[i, k]

        Vstore[:, :, VFiter] = V.reshape(sizez, sizek,)  # store value function at each
        # iteration for graphing later
        V = Vmat.max(axis=2)  # apply max operator to Vmat (to get V(k))
        PF = np.argmax(Vmat, axis=2)  # find the index of the optimal k'
        Vstore[:,:, i] = V  # store V at each iteration of VFI
        VFdist = (np.absolute(V - TV)).max()  # check distance between value
        # function for this iteration and value function from past iteration
        VFiter += 1
    VF = V
    return VF

In [16]:
### Solve the firm problem and return V and PF

In [19]:
VF = VFI_loop(w)

In [ ]:
VF

In [ ]:
e = cash_flow(w)

In [ ]:
np.shape(e)

In [ ]:
kgrid = kgrid_func(w)
sizek = kgrid.shape[0]
sizez = z_grid.shape[0]

In [ ]:
PF = VFI_loop(w)

In [ ]:
PF

In [ ]:
### Define market clear function
@numba.jit
def MKclear(w):
    
    kgrid = kgrid_func(w)
    
    e = cash_flow(w)
    
    sizek = kgrid.shape[0]
    sizez = z_grid.shape[0]


    ### Collect optimal values(functions)
    # Optimal capital stock k'
    PF = VFI_loop(w)
    optK = kgrid[PF]

    # optimal investment I
    optI = optK - (1 - delta) * kgrid

    # optimal labor demand
    optLD = np.zeros((sizez, sizek))
    for i in range(sizez):
        for j in range(sizek):
            optLD[i,j] = (((alpha_l / w) ** (1 / (1 - alpha_l))) *
          ((kgrid[j] ** alpha_k) ** (1 / (1 - alpha_l))) * (z_grid[i] ** (1/(1 - alpha_l))))


    ### Find Stationary Distribution
    Gamma = np.ones((sizez, sizek)) * (1 / (sizek * sizez))
    SDtol = 1e-12
    SDdist = 7
    SDiter = 0
    SDmaxiter = 1000
    while SDdist > SDtol and SDmaxiter > SDiter:
        HGamma = SD_loop(PF, Gamma, sizez, sizek)
        SDdist = (np.absolute(HGamma - Gamma)).max()
        Gamma = HGamma
        SDiter += 1
        
    ### Find aggregate values
    # labor demand
    optALD = Agg(optLD, Gamma)

    # Investment
    optAI = Agg(optI, Gamma)

    # Adjustment costs
    optADJC = psi/2 * np.multiply((optI)**2, 1/kgrid)
    optAADJC = Agg(optADJC, Gamma)

    # Output
    optY = np.multiply(np.multiply((optLD) ** alpha_l, kgrid ** alpha_k),np.transpose([z_grid]))
    optAY = Agg(optY, Gamma)

    # Consumption
    optCON = optAY - optAI - optAADJC


    ### Find labor supply
    optALS = w/(h * optCON)

    mkclear = abs(optALS - optALD)
    
    return mkclear
    
# Call the minimizer
# Minimize with (truncated) Newton's method (called Newton Conjugate Gradient method)
wage_initial = w
GE_results = opt.minimize(MKclear,  wage_initial, method='Nelder-Mead', tol = 1e-12, options={'maxiter': 5000})

In [ ]:
sigma_eps = 0.213

In [ ]:
num_draws = 100000 # number of shocks to draw
eps = np.random.normal(0.0, sigma_eps, size=(num_draws))

In [ ]:
np.shape(eps)